In [1]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
from scipy import misc
from skimage.io import imread_collection, imsave
from skimage import *
import cv2
from skimage.restoration import estimate_sigma

#creating a collection with the available images
noisy = imread_collection('dataset/Noisy/*.jpg')
blurry = imread_collection('dataset/Blurry/*.jpg')

In [ ]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def singleChannel(image, sig, vrange):
    blurred = ndimage.gaussian_filter(image, sigma=sig, mode='reflect')
    result = image + (image - blurred) * 1
    if vrange is not None:
        return np.clip(result, vrange[0], vrange[1], out=result)
    return result


def splitColor(image):
    red = image[:, :, 0]
    green = image[:, :, 1]
    blue = image[:, :, 2]
    return(red, green, blue)


def noisyTreatment(layers):
    arr = []
    k = np.array([[0,0,0],[0,1,0],[0,0,0]])
    k2 = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
    for layer in layers:
        gauss_denoised = ndimage.gaussian_filter(layer, sigma=2.5)
        layerFlou = scipy.ndimage.convolve(gauss_denoised, k)
        result = layer + (layer - layerFlou) * 1
        arr.append(result)

    return np.dstack((arr[0], arr[1], arr[2]))

from skimage.filters import unsharp_mask

def process_noisy(collection):  
    for file in range(len(collection)):
        image = data.moon()
        result = unsharp_mask(collection[file], radius=1, amount=1)
        imsave('dataset/noisyEnd/' + collection.files[file][-7:-4] + '.jpg', img_as_ubyte(result))
    return 1
        
def process_blurry(collection):  
    for file in range(len(collection)):
        image = data.moon()
        result = unsharp_mask(collection[file], radius=1, amount=1)
        imsave('dataset/blurryEnd/' + collection.files[file][-7:-4] + '.jpg', img_as_ubyte(result))
    return 1


In [2]:
#process_noisy(noisy)
#process_blurry(blurry)

In [3]:
# Get the Denoised images
noisyTreated = imread_collection('dataset/noisyEnd/*.jpg')

def estimate_noise(listNoise, listDeNoise):
    listSigmaNoise = []
    listSigmaDenoise = []
    # Get the noise sigma of each image then add it to the list
    for imageNoise in listNoise:
        sigmaNoise = estimate_sigma(imageNoise, multichannel=True, average_sigmas=True)
        listSigmaNoise.append(sigmaNoise)
    
    # Calculation of the mean sigma of the noised images
    totalSigmaNoise = sum(listSigmaNoise)
    meanSigmaNoise = totalSigmaNoise/len(listNoise)
    
    # Same action here for denoised images
    for imageDenoise in listDeNoise:
        sigmaDenoise = estimate_sigma(imageDenoise, multichannel=True, average_sigmas=True)
        listSigmaDenoise.append(sigmaDenoise)

    totalSigmaDenoise = sum(listSigmaDenoise)
    meanSigmaDenoise = totalSigmaDenoise/len(listDeNoise)

    return(meanSigmaNoise,meanSigmaDenoise)

# Calculation of the percentage difference between the sigma to obtain the performance index
meanSigmaNoise,meanSigmaDenoise = estimate_noise(noisy,noisyTreated)
performance_index_noisy = ((meanSigmaDenoise - meanSigmaNoise)/meanSigmaNoise)*100
print(round(performance_index_noisy,2))

39.91
